<a href="https://colab.research.google.com/github/kalyanchakravarthy18/blood-group-detection/blob/main/blood_group_model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import numpy as np
import cv2
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.utils import to_categorical
from google.colab import drive

# Mount Google Drive to access dataset
drive.mount('/content/drive')

# Set dataset path
dataset_path = '/content/drive/MyDrive/fingerprints'

# Load images and labels from the dataset
def load_images_and_labels(dataset_path):
    images = []
    labels = []
    for label in os.listdir(dataset_path):
        label_dir = os.path.join(dataset_path, label)
        if os.path.isdir(label_dir):  # Check if it’s a directory
            for image_file in os.listdir(label_dir):
                img_path = os.path.join(label_dir, image_file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img = cv2.resize(img, (128, 128))  # Resize to 128x128 pixels
                    images.append(img)
                    labels.append(label)  # Use folder name as label
    return np.array(images), np.array(labels)

# Load and preprocess the dataset
X, y = load_images_and_labels(dataset_path)
X = X / 255.0  # Normalize the pixel values
X = X.reshape(-1, 128, 128, 1)  # Add channel dimension

# Encode labels into categorical format
label_mapping = {label: idx for idx, label in enumerate(np.unique(y))}
y_encoded = np.array([label_mapping[label] for label in y])
y_categorical = to_categorical(y_encoded)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_mapping), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(rotation_range=10, zoom_range=0.1, width_shift_range=0.1, height_shift_range=0.1)
datagen.fit(X_train)

# Train the model
model.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_test, y_test), epochs=10)

# Evaluate the model
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=label_mapping.keys()))

# Function to predict blood group from a new fingerprint image
def predict_blood_group(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128)) / 255.0
    img = img.reshape(1, 128, 128, 1)
    prediction = np.argmax(model.predict(img), axis=1)[0]
    blood_group = [key for key, value in label_mapping.items() if value == prediction][0]
    return blood_group




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


151/151 ━━━━━━━━━━━━━━━━━━━━ 161s 1s/step - accuracy: 0.2237 - loss: 1.9541 - val_accuracy: 0.6753 - val_loss: 0.9445
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 209s 1s/step - accuracy: 0.4882 - loss: 1.3504 - val_accuracy: 0.7560 - val_loss: 0.6678
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 170s 1s/step - accuracy: 0.5684 - loss: 1.1241 - val_accuracy: 0.7086 - val_loss: 0.7206
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 199s 1s/step - accuracy: 0.6250 - loss: 1.0017 - val_accuracy: 0.7152 - val_loss: 0.7282
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 173s 1s/step - accuracy: 0.6576 - loss: 0.9241 - val_accuracy: 0.7619 - val_loss: 0.6229
Epoch 6/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 170s 1s/step - accuracy: 0.6686 - loss: 0.8799 - val_accuracy: 0.8018 - val_loss: 0.5857
Epoch 7/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 185s 1s/step - accuracy: 0.6794 - loss: 0.8547 - val_accuracy: 0.8451 - val_loss: 0.4310
Epoch 8/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.6903 - loss: 0.8627 - val_accuracy: 0.836

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
# Test prediction with a sample image
sample_image_path = '/content/drive/MyDrive/fingerprints/A-.BMP'  # Replace with path to your test image
try:
    print("Predicted Blood Group:", predict_blood_group(sample_image_path))
except ValueError as e:
    print(e)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted Blood Group: A-


In [ ]:
model.save('/content/drive/MyDrive/bloodgroup detection/detection/bloodgroup.keras')